In [15]:
import numpy as np
import scipy as sp
import pymongo
import pandas as pd
import nsepy
from datetime import datetime
from datetime import date
from datetime import timedelta
from pymongo import MongoClient
from nsepy import get_history

client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]

In [16]:
def getIndustryPctChange(stockName,industryName,startDate,endDate,client):
    
    db=client["stockWarehouse"]
    equityList=db["stocklist"]
    stockWarehouse=db["stockWarehouse"]
    #print(industryName,startDate,endDate)
    finaldf=pd.DataFrame(stockWarehouse.find({'Symbol':stockName,'Date':{'$gt':startDate},'Date':{'$lt':endDate}},{'_id':0,'Date':1,'Close':1}))
    finaldf['Date']=finaldf['Date'].astype('datetime64[ns]')
    finaldf.set_index('Date',inplace=True)
    finaldf.sort_index(inplace=True)
    for i in equityList.find({'Issuer Name':{'$ne':stockName},"ISIN No":industryName,'Security Name':{'$ne':'Inactive'},'State':"A"},{'Issuer Name':1,"_id":0}):
         tmpdf=pd.DataFrame(stockWarehouse.find({'Symbol':i['Issuer Name'],'Date':{'$gt':startDate},'Date':{'$lt':endDate}},{'_id':0,'Date':1,'Close':1}))
         tmpdf['Date']=tmpdf['Date'].astype('datetime64[ns]') 
         tmpdf.set_index('Date',inplace=True)
         tmpdf.sort_index(inplace=True)   
         finaldf=finaldf + tmpdf
    
    finaldf['industrypctchange1']=finaldf['Close'].pct_change(1)*1000
    finaldf['industrypctchange3']=finaldf['Close'].pct_change(3)*1000
    finaldf['industrypctchange5']=finaldf['Close'].pct_change(5)*1000
    finaldf.dropna(inplace=True)
    return finaldf        
            
    


In [17]:
stockName='SBIN'
industryName=0
startDate=0
endDate=0
# for i1 in stockWarehouse.find():
for i1 in stockWarehouse.find({'Symbol':stockName},{'_id':0,'Date':1}).sort('Date',1).limit(1):
    startDate=i1['Date']    
for i2 in stockWarehouse.find({'Symbol':stockName},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
    endDate=i2['Date']    

industrydf=pd.DataFrame(getIndustryPctChange(stockName,industryName,startDate,endDate,client))
industrydf.drop('Close',inplace=True,axis=1)


niftydf=pd.DataFrame(get_history(symbol="NIFTY",start=startDate,end=endDate,index=True))
niftydf.index=niftydf.index.astype('datetime64[ns]')
niftydf.sort_index(inplace=True)
niftydf.drop(['Open', 'High', 'Low', 'Volume', 'Turnover'],axis=1,inplace=True)
niftydf['niftypctchange1']=niftydf['Close'].pct_change(1)*1000
niftydf['niftypctchange3']=niftydf['Close'].pct_change(3)*1000
niftydf['niftypctchange5']=niftydf['Close'].pct_change(5)*1000
niftydf.dropna(inplace=True)
niftydf.drop('Close',inplace=True,axis=1)

stockdf=pd.DataFrame(stockWarehouse.find({'Symbol':stockName},{'_id':0,'Date':1,'Close':1}))
stockdf['Date']=stockdf['Date'].astype('datetime64[ns]')
stockdf.set_index('Date',inplace=True)
stockdf.sort_index(inplace=True)
outputdf=stockdf.copy()

stockdf['stockpctchange1']=stockdf['Close'].pct_change(1)*1000
stockdf['stockpctchange3']=stockdf['Close'].pct_change(3)*1000
stockdf['stockpctchange5']=stockdf['Close'].pct_change(5)*1000


stockdf['futurechange1']=stockdf['Close'].pct_change(-1)
stockdf.loc[stockdf['futurechange1'] <= 0, 'futurechange1'] = 0
stockdf.loc[stockdf['futurechange1'] > 0, 'futurechange1'] = 1

stockdf['futurechange3']=stockdf['Close'].pct_change(-3)
stockdf.loc[stockdf['futurechange3'] <= 0, 'futurechange3'] = 0
stockdf.loc[stockdf['futurechange3'] > 0, 'futurechange3'] = 1


stockdf['futurechange5']=stockdf['Close'].pct_change(-5)
stockdf.loc[stockdf['futurechange5'] <= 0, 'futurechange5'] = 0
stockdf.loc[stockdf['futurechange5'] > 0, 'futurechange5'] = 1


stockdf.drop('Close',inplace=True,axis=1)

inputdf=niftydf.join(industrydf,on='Date')
inputdf=inputdf.join(stockdf,on='Date')

inputdf.sort_index(inplace=True)
inputdf.dropna(inplace=True)
inputdf.to_csv('data.csv',header=False,index=False)